In [1]:
from collections import defaultdict
from dataclasses import dataclass
import os
import sys
import yaml
import time
import numpy as np
import scipy.io
import scipy.linalg
from src.common import NDArrayFloat
from src.linalg import get_numpy_eigenvalues

In [2]:
@dataclass
class Performance:
    time: float = 0.0
    relative_error: float = 0.0


In [13]:
def run_test_cases(
    path_to_homework: str, path_to_matrices: str
) -> dict[str, Performance]:
    matrix_filenames = []
    performance_by_matrix = defaultdict(Performance)
    with open(os.path.join(path_to_homework, "matrices.yaml"), "r") as f:
        matrix_filenames = yaml.safe_load(f)
    for i, matrix_filename in enumerate(matrix_filenames):
        print(f"Processing matrix {i+1} out of {len(matrix_filenames)}")
        A = scipy.io.mmread(os.path.join(path_to_matrices, matrix_filename)).todense().A
        perf = performance_by_matrix[matrix_filename]
        t1 = time.time()
        eigvals = get_all_eigenvalues(A)
        print(f'matrix {i+1} done ny algo!')
        eigvals_exact = list(get_numpy_eigenvalues(A))
        print(f'matrix {i+1} done ny numpy!')
        t2 = time.time()
        perf.time += t2 - t1
        eigvals_exact.sort()
        eigvals.sort()
        perf.relative_error = np.median(
            np.abs(eigvals_exact - eigvals) / np.abs(eigvals_exact)
        )
    return performance_by_matrix

In [4]:
def qr(A: NDArrayFloat) -> NDArrayFloat:
    n = A.shape[0]
    Q = np.zeros_like(A)
    R = np.zeros_like(A)
    W = A.copy()
    
    for j in range(n):
        w_j_norm = np.linalg.norm(W[: , j])
        Q[: , j ] = W[: , j] / w_j_norm # W[: , j] == w_j^j
        for i in range(j):
            R[i,j] = A[: , j] @ Q[:, i]
        a_j_norm = np.linalg.norm(A[:, j])
        R[j,j] = np.sqrt(a_j_norm**2 - np.sum(R[ :j , j] ** 2))
        for k in range(j+1,n):
            prod = W[: , k] @ Q[: , j]
            W[:, k ] = W[: , k] - prod * Q[:, j]
    return Q,R

def get_all_eigenvalues(A: NDArrayFloat) -> NDArrayFloat:
    A_k = A.copy()
    n_iters = 10
    for k in range(n_iters):
        Q, R = qr(A_k)
        A_k = R @ Q
    out = np.array(np.diag(A_k), dtype=np.float64)
    return out

# Result summary:
# Matrix: bp__1000.mtx.gz. Average time: 2.01e+01 seconds. Relative error: 9.20e-01
# Matrix: e05r0100.mtx.gz. Average time: 1.50e+00 seconds. Relative error: 1.27e-01
# Matrix: fs_541_1.mtx.gz. Average time: 7.83e+00 seconds. Relative error: 3.61e-05
# Matrix: fs_680_1.mtx.gz. Average time: 1.27e+01 seconds. Relative error: 4.65e-07
# Matrix: gre_1107.mtx.gz. Average time: 4.48e+01 seconds. Relative error: 7.48e-01
# Matrix: hor__131.mtx.gz. Average time: 5.70e+00 seconds. Relative error: 1.04e+00
# Matrix: impcol_c.mtx.gz. Average time: 4.51e-01 seconds. Relative error: 8.11e-01
# Matrix: impcol_d.mtx.gz. Average time: 5.08e+00 seconds. Relative error: 8.96e-01
# Matrix: impcol_e.mtx.gz. Average time: 1.25e+00 seconds. Relative error: 8.51e-01
# Matrix: jpwh_991.mtx.gz. Average time: 3.43e+01 seconds. Relative error: 1.50e-01
# Matrix: lns__511.mtx.gz. Average time: 6.66e+00 seconds. Relative error: nan
# Matrix: mahindas.mtx.gz. Average time: 6.35e+01 seconds. Relative error: nan
# Matrix: mcca.mtx.gz. Average time: 9.81e-01 seconds. Relative error: nan
# Matrix: mcfe.mtx.gz. Average time: 1.63e+01 seconds. Relative error: nan
# Matrix: nos5.mtx.gz. Average time: 5.27e+00 seconds. Relative error: 1.82e-01
# Matrix: orsirr_1.mtx.gz. Average time: 3.59e+01 seconds. Relative error: 3.01e-01

In [17]:
def power_iteration(A, x0, max_iter=1000, tol=1e-6):
  """
  Finds the dominant eigenvalue and eigenvector of a matrix.

  Args:
    A: The input matrix (numpy array).
    x0: Initial guess for the eigenvector (numpy array).
    max_iter: Maximum number of iterations.
    tol: Tolerance for convergence.

  Returns:
    eigenvalue: The dominant eigenvalue.
    eigenvector: The corresponding eigenvector.
  """

  x = x0 / np.linalg.norm(x0)
  for _ in range(max_iter):
    x_prev = x
    x = A @ x
    x = x / np.linalg.norm(x)
    eigenvalue = x.T @ A @ x
    if np.linalg.norm(x - x_prev) < tol:
      break
  return eigenvalue, x

def get_all_eigenvalues(A, max_iter=1000, tol=1e-6):
  """
  Finds all eigenvalues of a matrix using the Power Iteration method with deflation.

  Args:
    A: The input matrix (numpy array).
    max_iter: Maximum number of iterations for each Power Iteration.
    tol: Tolerance for convergence in Power Iteration.

  Returns:
    eigenvalues: A list of all eigenvalues.
  """
  n = A.shape[0]
  eigenvalues = []
  B = A.copy() # Work with a copy to avoid modifying the original matrix

  for _ in range(n):
    x0 = np.random.rand(n) # Random initial guess for the eigenvector
    eigenvalue, eigenvector = power_iteration(B, x0, max_iter, tol)
    eigenvalues.append(eigenvalue)

    # Deflation: Remove the found eigenvalue's contribution
    B = B - eigenvalue * np.outer(eigenvector, eigenvector) 
  print('power method with inflation')
  eigenvalues = np.array(eigenvalues, dtype=np.float64)
  return eigenvalues




# Result summary:
# Matrix: e05r0100.mtx.gz. Average time: 1.02e+01 seconds. Relative error: 3.72e-01
# Matrix: fs_541_1.mtx.gz. Average time: 1.09e+02 seconds. Relative error: 6.93e-05
# Matrix: hor__131.mtx.gz. Average time: 3.75e+01 seconds. Relative error: 3.11e-02
# Matrix: impcol_c.mtx.gz. Average time: 3.64e+00 seconds. Relative error: 1.00e+00
# Matrix: impcol_d.mtx.gz. Average time: 6.41e+01 seconds. Relative error: 1.00e+00
# Matrix: impcol_e.mtx.gz. Average time: 1.07e+01 seconds. Relative error: 1.00e+00
# Matrix: lns__511.mtx.gz. Average time: 8.92e+01 seconds. Relative error: 1.00e+00
# Matrix: mcca.mtx.gz. Average time: 6.27e+00 seconds. Relative error: 6.52e-01
# Matrix: nos5.mtx.gz. Average time: 5.56e+01 seconds. Relative error: 1.31e-05

In [21]:
def get_all_eigenvalues(A, tol=1e-6, max_iter=500):
    """
    Finds all eigenvalues of a real symmetric matrix using the Jacobi method.

    Args:
        A: The input symmetric matrix (numpy array).
        tol: Tolerance for off-diagonal elements.
        max_iter: Maximum number of iterations.

    Returns:
        eigenvalues: A numpy array of eigenvalues.
    """

    n = A.shape[0]
    eigenvalues = np.diag(A).copy() # Initial eigenvalues are the diagonal elements

    for _ in range(max_iter):
        # Find the largest off-diagonal element in magnitude
        max_off_diag = 0
        p, q = 0, 0
        for i in range(n):
            for j in range(i + 1, n):
                if abs(A[i, j]) > max_off_diag:
                    max_off_diag = abs(A[i, j])
                    p, q = i, j

        if max_off_diag < tol: # Convergence check
            break

        # Calculate rotation angle
        theta = 0.5 * np.arctan2(2 * A[p, q], A[q, q] - A[p, p])

        # Construct rotation matrix
        R = np.eye(n)
        R[p, p] = np.cos(theta)
        R[q, q] = np.cos(theta)
        R[p, q] = -np.sin(theta)
        R[q, p] = np.sin(theta)

        # Apply rotation
        A = R.T @ A @ R
        eigenvalues = np.diag(A)
    eigenvalues = np.array(eigenvalues, dtype=np.float64)
    print("Jacobi method")
    eigenvalues = np.array(eigenvalues, dtype=np.float64)
    return eigenvalues


# Matrix: e05r0100.mtx.gz. Average time: 2.75e+00 seconds. Relative error: 3.44e-01
# Matrix: fs_541_1.mtx.gz. Average time: 1.48e+01 seconds. Relative error: 2.68e-04
# Matrix: hor__131.mtx.gz. Average time: 1.09e+01 seconds. Relative error: 1.37e-01
# Matrix: impcol_c.mtx.gz. Average time: 1.83e+00 seconds. Relative error: 6.85e-01
# Matrix: impcol_d.mtx.gz. Average time: 1.06e+01 seconds. Relative error: 7.81e-01
# Matrix: impcol_e.mtx.gz. Average time: 2.41e+00 seconds. Relative error: 1.00e+00
# Matrix: lns__511.mtx.gz. Average time: 1.39e+01 seconds. Relative error: 1.00e+00
# Matrix: mcca.mtx.gz. Average time: 1.73e+00 seconds. Relative error: 9.28e-02
# Matrix: nos5.mtx.gz. Average time: 1.01e+01 seconds. Relative error: 5.48e-02

In [37]:
def get_all_eigenvalues(A, max_iter=3000):
    """
    Finds all eigenvalues of a symmetric matrix using the QR algorithm with shifts.

    Args:
        A: The input symmetric matrix (numpy array).
        max_iter: Maximum number of iterations.
        tol: Tolerance for convergence.

    Returns:
        eigenvalues: A numpy array of eigenvalues.
    """

    Ak = A.copy()
    n = Ak.shape[0]

    for _ in range(max_iter):
        # Wilkinson shift
        d = (Ak[n-2, n-2] - Ak[n-1, n-1]) / 2
        mu = Ak[n-1, n-1] - Ak[n-1, n-2]**2 / (d + np.sign(d) * np.sqrt(d**2 + Ak[n-1, n-2]**2))

        Qk, Rk = np.linalg.qr(Ak - mu * np.eye(n))
        Ak = Rk @ Qk + mu * np.eye(n)

    eigenvalues = np.diag(Ak) 
    print("QR algorithm with shifts")
    eigenvalues = np.array(eigenvalues, dtype=np.float64)
    return eigenvalues 

# Matrix: nos5.mtx.gz. Average time: 4.99e+01 seconds. Relative error: 3.44e-02

In [27]:
def rayleigh_quotient_iteration(A, x0, max_iter=1000, tol=1e-6):
    """
    Finds an eigenvalue and eigenvector using the Rayleigh Quotient Iteration.

    Args:
        A: The input symmetric matrix (numpy array).
        x0: Initial guess for the eigenvector (numpy array).
        max_iter: Maximum number of iterations.
        tol: Tolerance for convergence.

    Returns:
        eigenvalue: The estimated eigenvalue.
        eigenvector: The corresponding eigenvector.
    """

    x = x0 / np.linalg.norm(x0)
    eigenvalue = x.T @ A @ x 

    for _ in range(max_iter):
        x_prev = x
        x = np.linalg.solve(A - eigenvalue * np.eye(A.shape[0]), x_prev) 
        x = x / np.linalg.norm(x)
        eigenvalue = x.T @ A @ x

        if np.linalg.norm(x - x_prev) < tol:
            break

    return eigenvalue, x


def get_all_eigenvalues(A, max_iter=1000, tol=1e-6):
    """
    Finds all eigenvalues of a symmetric matrix using Rayleigh Quotient Iteration with deflation.

    Args:
        A: The input symmetric matrix (numpy array).
        max_iter: Maximum number of iterations for each Rayleigh Quotient Iteration.
        tol: Tolerance for convergence in Rayleigh Quotient Iteration.

    Returns:
        eigenvalues: A list of all eigenvalues.
    """

    n = A.shape[0]
    eigenvalues = []
    B = A.copy()

    for _ in range(n):
        x0 = np.random.rand(n)
        eigenvalue, eigenvector = rayleigh_quotient_iteration(B, x0, max_iter, tol)
        eigenvalues.append(eigenvalue)

        # Deflation: 
        B = B - eigenvalue * np.outer(eigenvector, eigenvector)
    print("Rayleigh Quotient Iteration with deflation")
    eigenvalues = np.array(eigenvalues, dtype=np.float64)
    return eigenvalues

# Matrix: nos5.mtx.gz. Average time: 7.86e+02 seconds. Relative error: 7.28e-01

In [38]:
path_to_homework = os.path.join("")
path_to_matrices = os.path.join("matrices")
performance_by_matrix = run_test_cases(
    path_to_homework=path_to_homework,
    path_to_matrices=path_to_matrices,
)

print("\nResult summary:")
for filename, perf in performance_by_matrix.items():
    print(
        f"Matrix: {filename}. "
        f"Average time: {perf.time:.2e} seconds. "
        f"Relative error: {perf.relative_error:.2e}"
    )

Processing matrix 1 out of 1
QR algorithm with shifts
matrix 1 done ny algo!
matrix 1 done ny numpy!

Result summary:
Matrix: nos5.mtx.gz. Average time: 7.67e+01 seconds. Relative error: 2.66e-03
